# The effects of precision

Examining how varying precision (inverse variance) levels on the likelihood and prior affect the posterior distribution

==========================================================================

* **Notebook dependencies**:
    * ...

* **Content**: Jupyter notebook accompanying Chapter 2 of the textbook "Fundamentals of Active Inference"

* **Author**: Sanjeev Namjoshi (sanjeev.namjoshi@gmail.com)

* **Version**: 0.1

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from scipy.stats import norm
from types import SimpleNamespace

mpl.style.use("seaborn-deep")

The last example demonstrated how to perform Bayesian inference with a probabilistic generating function and Gaussian prior. In this example we examine the effects of how different levels of variance (or the inverse of variance, precision) can affect the Bayesian inference process. This will become very important in future examples of active inference agents whose behavior is very sensitive to levels of precision/variance.